# Plot some results from simulation 3

In [ ]:
"""
Plotting simulation results

Authors: Milind Kumar Vaddiraju, ChatGPT, Copilot
"""

# Necessary imports
import copy
from datetime import datetime
import json
# %matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import os
import pickle
import sys

sys.path.insert(0, os.path.abspath('..'))

from network_classes import *
from utils import *


In [ ]:
# Some common functions

def return_folder_paths(experiment_foldername):
    folder_names = []
    for folder in os.listdir(experiment_foldername):
        # Check if the folder is a directory
        if os.path.isdir(experiment_foldername + folder + "/"):
            folder_names.append(experiment_foldername + folder + "/")
            
    return folder_names

def obtain_plot_information(experiment_key, plotting_information, data_key):
        y = []
        x = []

        results_allUEs_per_lambda_contention = plotting_information[experiment_key]["results_allUEs_per_lambda_contention"]
        for lambda_value in results_allUEs_per_lambda_contention:
                x.append(lambda_value)
                y.append(\
                       results_allUEs_per_lambda_contention[lambda_value][data_key])
                
        y = [i for _, i in sorted(zip(x,y))]
        x = sorted(x)

        return (x, y)


def extract_plotting_data(folder_names, key_accessor):
    plotting_data_temp = {}
    for folder in folder_names:
        experiment_filename = folder + "experiment_parameters.pkl"
        with open(experiment_filename, "rb") as file:
            data = pickle.load(file)
        key = key_accessor(data)
        # print(key)
        if key in plotting_data_temp:
            plotting_data_temp[key]["lambda_range"] = np.concatenate((plotting_data_temp[key]["lambda_range"], data["experiment_parameters"]["lambda_range"]))
            plotting_data_temp[key]["results_allUEs_per_lambda_contention"].update(data["results_allUEs_per_lambda_contention"])
            # assert plotting_data_temp[key]["schedule_contention"] == data["schedule_contention"] TODO: Create equality for the Schedule class
            assert plotting_data_temp[key]["percentile_to_plot"] == data["experiment_parameters"]["percentile_to_plot"]
        else:
            plotting_data_temp[key] = {
                "lambda_range": data["experiment_parameters"]["lambda_range"],
                "results_allUEs_per_lambda_contention": data["results_allUEs_per_lambda_contention"],
                "schedule_contention": data["schedule_contention"],
                "percentile_to_plot": data["experiment_parameters"]["percentile_to_plot"],
                "num_UEs": data["experiment_parameters"]["num_UEs"]
            }
    return plotting_data_temp



In [ ]:
# Obtaining simulation duration


experiment_foldername = "../results/simulation_4/10UEs_varying_qbv_window/"

# # Generate folder names by iterating through the folders in the experiment folder
# folder_names = []
# for folder in os.listdir(experiment_foldername):
#     # Check if the folder is a directory
#     if os.path.isdir(experiment_foldername + folder + "/"):
#         folder_names.append(experiment_foldername + folder + "/")
folder_names = return_folder_paths(experiment_foldername)

print(folder_names)

total_duration = 0
for folder in folder_names:
    experiment_filename = folder + "experiment_parameters.json"
    with open(experiment_filename, "rb") as file:
        data = json.load(file)
    duration = data["execution_duration"]
    if duration < 10000:
        total_duration += duration
    print("Duration: ", duration)

print("Total duration: ", total_duration/60, " minutes")
print("Total duration: ", total_duration/3600, " hours")

# Plotting for a single simulation time

In [ ]:
label_prefix = {}
plotting_data = {}
plotting_keys = {}

In [ ]:
# Plotting basic CSMA results with 10 UEs and different aggregation sizes

# folder paths

experiment_foldername = "../results/simulation_3/10_UEs_varying_bus_size_964B_80MHz/"
folder_names = return_folder_paths(experiment_foldername)
print(folder_names)
print(len(folder_names))

plotting_data_temp = extract_plotting_data(folder_names, \
                                           lambda data: data["experiment_parameters"]["aggregation_limit"])

label_prefix["csma_964B_80MHz"] = "CSMA, aggregation limit =  "
plotting_data["csma_964B_80MHz"] = plotting_data_temp


In [ ]:
# Plotting roundrobin results with different qbv window sizes
# folder paths

experiment_foldername = "../results/simulation_4/10UEs_roundrobin_964B_80MHz/"
folder_names = return_folder_paths(experiment_foldername)



plotting_data_temp = extract_plotting_data(\
    folder_names, \
    lambda data: data["experiment_parameters"]["schedule_config"]["qbv_window_size"])


print(folder_names)
print(len(folder_names))


label_prefix["roundrobin_964B_80MHz"] = "rr, qbv window size (us) =  "
plotting_data["roundrobin_964B_80MHz"] = plotting_data_temp           


In [ ]:
# Plotting dynamic rr results with different packet sizes


experiment_foldername = "../results/simulation5/10UEs_dynamic_rr/"
folder_names = return_folder_paths(experiment_foldername)

folder_names.remove("../results/simulation5/10UEs_dynamic_rr/2024_05_19_19_37_32/")
folder_names.remove('../results/simulation5/10UEs_dynamic_rr/2024_05_19_11_43_16/')

print(folder_names)
print(len(folder_names))

plotting_data_temp = extract_plotting_data(folder_names, \
                                           lambda data: data["experiment_parameters"]["packet_sizes"][0])

label_prefix["dynamic_rr"] = "dynamic rr, packet size = "
plotting_data["dynamic_rr"] = plotting_data_temp

In [ ]:
# Plotting dynamic rr results with max weight scheduling


experiment_foldername = "../results/simulation5/10UEs_dynamic_max_weight/"
folder_names = return_folder_paths(experiment_foldername)


print(folder_names)
print(len(folder_names))

plotting_data_temp = extract_plotting_data(folder_names, \
                                           lambda data: data["experiment_parameters"]["packet_sizes"][0])

label_prefix["dynamic_max_weight"] = "dynamic max weight, packet size = "
plotting_data["dynamic_max_weight"] = plotting_data_temp

In [ ]:
# Plotting dynamic rr results with max weight scheduling


experiment_foldername = "../results/simulation5/10UEs_dynamic_grr/"
folder_names = return_folder_paths(experiment_foldername)


print(folder_names)
print(len(folder_names))

plotting_data_temp = extract_plotting_data(folder_names, \
                                           lambda data: data["experiment_parameters"]["schedule_config"]["scaling_factor"])

label_prefix["dynamic_grr_2UEs"] = "dynamic_grr, 2UEs, scaling factor ="
plotting_data["dynamic_grr_2UEs"] = plotting_data_temp

In [ ]:
# Plotting max weight scheduling with different qbv_window sizes

# folder paths

# experiment_foldername = "../results/simulation_4/10UEs_roundrobin_blank_964B_80MHz/"
experiment_foldername = "../results/simulation_4/10UEs_max_weight_rr/"
folder_names = return_folder_paths(experiment_foldername)
print(folder_names)
print(len(folder_names))

plotting_data_temp = extract_plotting_data(folder_names, \
                                           lambda data: data["experiment_parameters"]["schedule_config"]["qbv_window_size"])

label_prefix["max_weight_rr"] = "max_wght_rr, qbv_win_size (us) =  "
plotting_data["max_weight_rr"] = plotting_data_temp


In [ ]:
# Plotting 2 UEs grouped roundrobin results with different number of UEs together


experiment_foldername = "../results/simulation_4/10UEs_grouped_roundrobin_964B_80MHz/num_UEs_together_2/"
folder_names = return_folder_paths(experiment_foldername)
print(folder_names)
print(len(folder_names))


plotting_data_temp = extract_plotting_data(folder_names, \
                                           lambda data: data["experiment_parameters"]["schedule_config"]["qbv_window_size"])

label_prefix["grouped_roundrobin_2UEs"] = "grr, num UEs tgthr = 2, qbv win size (us) ="
plotting_data["grouped_roundrobin_2UEs"] = plotting_data_temp

In [ ]:
# Plotting schedule 5 with different parameters
experiment_foldername = "../results/simulation_4/10UEs_schedule5/"
folder_names = return_folder_paths(experiment_foldername)
print(folder_names)
print(len(folder_names))

plotting_data_temp = extract_plotting_data(folder_names, \
                                           lambda data: str(data["experiment_parameters"]["schedule_config"]["qbv_window_size"]) + \
                                            ", " + str(data["experiment_parameters"]["schedule_config"]["contention_window_size"]) + \
                                            ", " + str(data["experiment_parameters"]["schedule_config"]["num_UEs_together_contention"]))

label_prefix["schedule5"] = "qbv_wind, cont_wind, n_UEs_tgthr_contention = "
plotting_data["schedule5"] = plotting_data_temp

In [ ]:
# Plotting schedule "rr then partial contention" with different parameters
experiment_foldername = "../results/simulation_4/10UEs_rr_pc/"
folder_names = return_folder_paths(experiment_foldername)
print(folder_names)
print(len(folder_names))

plotting_data_temp = extract_plotting_data(folder_names, \
                                           lambda data: str(data["experiment_parameters"]["schedule_config"]["qbv_window_size"]) + \
                                            ", " + str(data["experiment_parameters"]["schedule_config"]["contention_window_size"]) + \
                                            ", " + str(data["experiment_parameters"]["schedule_config"]["contention_UE_indices"]))

label_prefix["10UEs_rr_pc"] = "qw, cw, cUEi = "
plotting_data["10UEs_rr_pc"] = plotting_data_temp

In [ ]:
# Plotting schedule "rr then partial contention" with different parameters
experiment_foldername = "../results/simulation_4/10UEs_grr/2UEs/"
folder_names = return_folder_paths(experiment_foldername)
print(folder_names)
print(len(folder_names))

plotting_data_temp = extract_plotting_data(folder_names, \
                                           lambda data: str(data["experiment_parameters"]["schedule_config"]["qbv_window_size"]) + \
                                            ", " + str(data["experiment_parameters"]["schedule_config"]["offset"]) + \
                                            ", " + str(data["experiment_parameters"]["schedule_config"]["num_UEs_together"]))

label_prefix["10UEs_grr"] = "(grr) qw, offset, num_UE = "
plotting_data["10UEs_grr"] = plotting_data_temp

In [ ]:
# Plotting schedule 3 with different contention window sizes for 1 UE together
experiment_foldername = "../results/simulation_4/10UEs_schedule3/num_together_1/"
folder_names = return_folder_paths(experiment_foldername)
print(folder_names)
print(len(folder_names))

plotting_data_temp = extract_plotting_data(folder_names, \
                                           lambda data: data["experiment_parameters"]["schedule_config"]["contention_window_size"])

label_prefix["schedule3_1500_x_1"] = "qbv = 1500us, qbv_n_UEs = 1, contention_slot =  "
plotting_data["schedule3_1500_x_1"] = plotting_data_temp

In [ ]:
# Plotting CSMA with different aggregation lengths for 10 UEs for 964 B payload size
experiment_foldername = "../results/simulation_3/10_UEs_varying_bus_size_DLMU_964B/"
folder_names = return_folder_paths(experiment_foldername)
print(folder_names)
print(len(folder_names))

plotting_data_temp = extract_plotting_data(folder_names, \
                                           lambda data: data["experiment_parameters"]["aggregation_limit"])

key = "csma_964B"
label_prefix[key] = "CSMA, 964B, aggr =  "
plotting_data[key] = plotting_data_temp

In [ ]:
# experiment_foldername = "../results/simulation_4/10UEs_roundrobin_vs_csma/"
# experiment_foldername = "../results/simulation_4/10UEs_grouped_roundrobin/1500/"
# experiment_foldername = "../results/simulation_4/10UEs_grouped_roundrobin/2000/"
# experiment_foldername = "../results/simulation_4/10UEs_schedule3/"
experiment_foldername = "../results/simulation_4/extras/"

os.makedirs(experiment_foldername, exist_ok=True)
roundrobin_keys = sorted(plotting_data["roundrobin_964B_80MHz"].keys())
roundrobin_keys = [1500]
# roundrobin_keys = []
csma_keys = sorted(plotting_data["csma_964B_80MHz"].keys())
csma_keys = [20, 30]
# csma_keys = []
# grouped_roundrobin_1500_keys = sorted(plotting_data["grouped_roundrobin_1500"].keys())
grouped_roundrobin_1500_keys = [1,2,3,4]
grouped_roundrobin_1500_keys = []
# grouped_roundrobin_2000_keys = sorted(plotting_data["grouped_roundrobin_2000"].keys())
# grouped_roundrobin_2000_keys = [2,3,4,5,6,7]
grouped_roundrobin_2000_keys = []
# schedule3_1500_x_1_keys = sorted(plotting_data["schedule3_1500_x_1"].keys())
schedule3_1500_x_1_keys = []

plotting_keys["csma_964B_80MHz"] = csma_keys
plotting_keys["roundrobin_964B_80MHz"] = roundrobin_keys
plotting_keys["grouped_roundrobin_1500"] = grouped_roundrobin_1500_keys
plotting_keys["grouped_roundrobin_2000"] = grouped_roundrobin_2000_keys
plotting_keys["schedule3_1500_x_1"] = schedule3_1500_x_1_keys
plotting_keys["csma_964B"] = []#sorted(plotting_data["csma_964B"].keys())
plotting_keys["schedule4"] = []#["690, 690, 2"]#sorted(plotting_data["schedule4"].keys())
plotting_keys["dynamic_rr"] = [64]#[64]
plotting_keys["dynamic_max_weight"] = [964]
plotting_keys["max_weight_rr"] = []# sorted(plotting_data["max_weight_rr"].keys())#[1250]#[690, 1000, 2000]#
plotting_keys["grouped_roundrobin_2UEs"] = []#sorted(plotting_data["grouped_roundrobin_2UEs"].keys())
plotting_keys["dynamic_grr_2UEs"] = sorted(plotting_data["dynamic_grr_2UEs"].keys())


plotting_keys["schedule5"] = []
for qbv_window_size in []:
        for contention_window_size in [690, 1000, 1250, 1500, 2000, 2500]:
                for num_UEs_together_contention in [1,2,3,4]:
                        plotting_keys["schedule5"].append(str(qbv_window_size) + ", " + str(contention_window_size) + ", " + str(num_UEs_together_contention))

plotting_key_experiment_temp = "10UEs_rr_pc"
plotting_keys[plotting_key_experiment_temp] = []
for qbv_window_size in []:#[690, 1000, 1250, 1500, 2000, 2500]:
        for contention_window_size in range(700, 2600, 100):
                for num_UEs_together_contention in [[4,5,6],[3,4,5,6], [3,4,5,6,7],[2,3,4,5,6,7]]:
                        plotting_keys[plotting_key_experiment_temp].append(str(qbv_window_size) + ", " + str(contention_window_size) + ", " + str(num_UEs_together_contention))


plotting_key_experiment_temp = "10UEs_grr"
plotting_keys[plotting_key_experiment_temp] = []
for qbv_window_size in [800, 1500, 2500]:#[690, 800, 1000, 1100, 1250, 1500, 1700, 1900, 2000, 2200, 2500]:
        for offset in [5]:#range(1,10):
                for num_UEs_together_contention in [2]:
                        plotting_keys[plotting_key_experiment_temp].append(str(qbv_window_size) + ", " + str(offset) + ", " + str(num_UEs_together_contention))




plot_linewidths = {}
plot_linewidths["dynamic_rr"] = 4
plot_linewidths["roundrobin_964B_80MHz"] = 4


save_file = False

scale = "linear"
percentile_to_plot = 99
percentile_filename = "percentile_latency_zoomed_" + scale + ".png"
percentile_slope_filename = "percentile_slope_zoomed_" + scale + ".png"
mean_filename = "mean_latency_zoomed_" + scale + ".png"
mean_slope_filename = "mean_slope_zoomed_" + scale + ".png"
n_packets_not_served_filename = "n_packets_not_served_zoomed_" + scale + ".png"
bus_occupancy_filename = "bus_occupancy_zoomed_" + scale + ".png"
n_wins_filename = "n_wins_zoomed_" + scale + ".png"
queue_slope_filename = "queue_slope_zoomed_" + scale + ".png"
# label_prefix_temp = "number of STAs = "

scaling_factor = 1e6


##### Generate color schemes

total_lines = sum(len(plotting_keys[schedule_key]) for schedule_key in label_prefix)

# Generate unique colors
colors = cm.get_cmap('tab20', total_lines)


##################### Plot the  percentile latency curve #####################
line_index = 0

plt.figure(figsize=(10, 8))
for schedule_key in label_prefix:
        label_prefix_temp = label_prefix[schedule_key]
        if schedule_key in plot_linewidths:
                linewidth = plot_linewidths[schedule_key]
        else:
                linewidth = 2
        for plot_key in plotting_keys[schedule_key]:
                lambda_range, y_values = obtain_plot_information(plot_key, plotting_data[schedule_key], "percentile_latency")
                plt.plot(np.array(lambda_range)*scaling_factor, \
                        np.array(y_values)/1e3, ".-", label = label_prefix_temp + str(plot_key),\
                        linewidth=linewidth, color=colors(line_index))
                line_index += 1


plt.xlabel("$\lambda$ (packets/s)", fontsize=15)
plt.ylabel("$99^{th}$ percentile latency (ms)", fontsize=15)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(prop={'size': 10})
plt.grid()

if scale == "linear":
        plt.ylim(bottom=0)
        plt.ylim(top = 40)
        # plt.xlim(0,2000)
# plt.xlim(left=0)
# plt.ylim(0,50000)


if scale == "log":
        plt.yscale('log')
        plt.xscale('log')

title = ("$99^{th}$ percentile latency vs load\n" 
        )
plt.title(title, fontsize=18)
# Insert a textbox at the lowest y value of the plot and have y axis be the label

plt.tight_layout()
if save_file:
        plt.savefig(os.path.join(experiment_foldername, percentile_filename))
plt.show()



##################### Plot the  mean latency curve #####################
line_index = 0
plt.figure(figsize=(10, 8))


for schedule_key in label_prefix:
        label_prefix_temp = label_prefix[schedule_key]
        if schedule_key in plot_linewidths:
                linewidth = plot_linewidths[schedule_key]
        else:
                linewidth = 2
        for plot_key in plotting_keys[schedule_key]:
                lambda_range, y_values = obtain_plot_information(plot_key, plotting_data[schedule_key], "mean_latency")
                plt.plot(np.array(lambda_range)*scaling_factor, \
                        np.array(y_values)/1e3, ".-", label = label_prefix_temp + str(plot_key),\
                        linewidth=linewidth, color=colors(line_index))
                line_index += 1

plt.xlabel("lambda (packets/s)")
plt.ylabel("Mean latency (ms)")
plt.legend(prop={'size': 10})
plt.grid()
plt.ylim(0,40)
# plt.xlim(0,1000)


if scale == "log":
        plt.yscale('log')

title = (f"Mean latency vs lambda,\n" 
        )
plt.title(title)
# Insert a textbox at the lowest y value of the plot and have y axis be the label

plt.tight_layout()
if save_file:
        plt.savefig(os.path.join(experiment_foldername, mean_filename))
plt.show()



##################### Plot the number of unserved packets #####################
line_index = 0
plt.figure(figsize=(10, 8))

for schedule_key in label_prefix:
        label_prefix_temp = label_prefix[schedule_key]
        for plot_key in plotting_keys[schedule_key]:
                lambda_range, y_values = obtain_plot_information(plot_key, plotting_data[schedule_key], "n_packets_not_served")
                plt.plot(np.array(lambda_range)*scaling_factor, \
                        np.array(y_values)/plotting_data[schedule_key][plot_key]["num_UEs"], ".-", label = label_prefix_temp + str(plot_key),\
                        linewidth=2, color=colors(line_index))
                line_index += 1
# plt.plot(n_packets_generated, percentiles)
plt.xlabel("lambda (packets/s)")
plt.ylabel("Unserved packets")
plt.legend(prop={'size': 10})
plt.grid()

if scale == "log":
        plt.yscale('log')

title = (f"Unserved packets vs lambda,\n" 
        )
plt.title(title)
# Insert a textbox at the lowest y value of the plot and have y axis be the label

plt.tight_layout()
if save_file:
        plt.savefig(os.path.join(experiment_foldername, n_packets_not_served_filename))
plt.show()

In [ ]:
# # Plotting bus sizes/contention wins/queue slope

# for key in sorted(plotting_data.keys()):

#         bus_occupancy_contention = []
#         bus_occupancy_contention_std = []
#         # lambda_range = plotting_data[key]["lambda_range"]
#         lambda_range = []
#         # for lambda_value in plotting_data[key]["results_allUEs_per_lambda_contention"]:
#         #         lambda_range.append(lambda_value)

#         results_allUEs_per_lambda_contention = plotting_data[key]["results_allUEs_per_lambda_contention"]
#         for lambda_value in results_allUEs_per_lambda_contention:
#                 lambda_range.append(lambda_value)
#                 bus_occupancy_contention.append(\
#                        results_allUEs_per_lambda_contention[lambda_value]["bus_occupancy"])


#         # sort bus_occupancy_contention and lambda_range and by lambda range
#         bus_occupancy_contention = [x for _, x in sorted(zip(lambda_range, bus_occupancy_contention))]
#         bus_occupancy_contention_std = [x for _, x in sorted(zip(lambda_range, bus_occupancy_contention_std))]
#         lambda_range = sorted(lambda_range)

#         plt.plot(np.array(lambda_range)*scaling_factor, \
#                 bus_occupancy_contention, ".-", label = label_prefix + str(key))
#         # plt.plot(np.array(lambda_range)*(schedule_contention.end_time - schedule_contention.start_time), \
#         #          mean_latencies_contention, ".-", label = "num_UEs = " + str(key))
# # plt.plot(n_packets_generated, percentiles)
# plt.xlabel("lambda (packets/s)")
# plt.ylabel("Bus occupancy")
# plt.legend()


# if scale == "log":
#         plt.yscale('log')

# title = (f"Simulation 3 bus occupancy vs lambda,\n" 
#         )
# plt.title(title)
# # Insert a textbox at the lowest y value of the plot and have y axis be the label

# plt.tight_layout()
# plt.savefig(os.path.join(experiment_foldername, bus_occupancy_filename))
# plt.show()






# for key in sorted(plotting_data.keys()):

#         n_wins_contention = []
#         n_wins_contention_std = []
#         # lambda_range = plotting_data[key]["lambda_range"]
#         lambda_range = []
#         # for lambda_value in plotting_data[key]["results_allUEs_per_lambda_contention"]:
#         #         lambda_range.append(lambda_value)

#         results_allUEs_per_lambda_contention = plotting_data[key]["results_allUEs_per_lambda_contention"]
#         for lambda_value in results_allUEs_per_lambda_contention:
#                 lambda_range.append(lambda_value)
#                 n_wins_contention.append(\
#                        results_allUEs_per_lambda_contention[lambda_value]["contention_wins"])


#         # sort n_wins_contention and lambda_range and by lambda range
#         n_wins_contention = [x for _, x in sorted(zip(lambda_range, n_wins_contention))]
#         n_wins_contention_std = [x for _, x in sorted(zip(lambda_range, n_wins_contention_std))]
#         lambda_range = sorted(lambda_range)
#         print(n_wins_contention)

#         plt.plot(np.array(lambda_range)*scaling_factor, \
#                 n_wins_contention, ".-", label = label_prefix + str(key))
#         # plt.plot(np.array(lambda_range)*(schedule_contention.end_time - schedule_contention.start_time), \
#         #          mean_latencies_contention, ".-", label = "num_UEs = " + str(key))
# # plt.plot(n_packets_generated, percentiles)
# plt.xlabel("lambda")
# plt.ylabel("Num wins")
# plt.legend()


# if scale == "log":
#         plt.yscale('log')

# title = (f"Simulation 3 n_wins vs lambda,\n" 
#         )
# plt.title(title)
# # Insert a textbox at the lowest y value of the plot and have y axis be the label

# plt.tight_layout()
# plt.savefig(os.path.join(experiment_foldername, n_wins_filename))
# plt.show()

plt.figure(figsize=(10, 8))

for schedule_key in label_prefix:
        label_prefix_temp = label_prefix[schedule_key]
        for plot_key in plotting_keys[schedule_key]:
                lambda_range, y_values = obtain_plot_information(plot_key, plotting_data[schedule_key], "queue_slope")
                plt.plot(np.array(lambda_range)*scaling_factor, \
                        np.array(y_values)*10**6, ".-", label = label_prefix_temp + str(plot_key),\
                        linewidth=2)

# plt.plot(n_packets_generated, percentiles)
plt.xlabel("lambda (packets/s)")
plt.ylabel("queue slope")
plt.legend(prop={'size': 10})
plt.grid()

if scale == "log":
        plt.yscale('log')

title = (f"Queue slope vs lambda,\n" 
        )
plt.title(title)
# Insert a textbox at the lowest y value of the plot and have y axis be the label

plt.tight_layout()
if save_file:
        plt.savefig(os.path.join(experiment_foldername, n_packets_not_served_filename))
plt.show()


